# Optuna

Optuna — фреймворк для оптимизации поиска оптимальных гиперпараметров модели.

В отличие от всем известного GridSearch'a Оптуна не перебирает все комбинации параметров, а использует более умный подход.

Чтобы понять, как ее использовать, давайте разберем ее по частям.


In [ ]:
!pip install optuna catboost -qqq
import optuna
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme()
from tqdm.notebook import tqdm

from sklearn.model_selection import KFold
from catboost import Pool, CatBoostClassifier

## 1. Objective function
В этой функции нужно написать код подсчета метрики, которую возвращаем. Objective вызывается Отпуной много раз для подбора лучших параметров.



In [ ]:
​def objective(trial, ...):
    # calculate score...
    return score

## 2. Trial object
В trial мы передаем параметры для "перебора", используя для каждого типа свой метод.

К примеру,  trial.suggest_float('x', -10, 10) где метод suggest_float показывает, что перебираем в float значениях, а -10 и 10 стартовый и конечные шаги.


In [ ]:
# Категориальное значение
loss_function = trial.suggest_categorical('loss', ['Logloss', 'CrossEntropy'])

# Целочисленное значение
depth = trial.suggest_int('depth', 5, 8)

# Равномерное распределение
learning_rate = trial.suggest_uniform('learning_rate', 0.0, 1.0)

## 3. Study parameter
Инициализируем обьект **study**, который начнет перебор и сохранит в себе историю результатов.
Если мы стараемя увеличить метрику, а не уменьшить ошибку, то используем `create_study(direction='maximize')` 

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=10)

## Итоговый код может выглядеть подобным образом:

In [ ]:
import optuna

def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study()
study.optimize(objective, n_trials=100)
study.best_params  # E.g. {'x': 2.002108042}

## Давайте запустим Optuna на наших данных

Подгрузим данные и отберем колонки

In [ ]:
!pip install kaggle -qqq
!mkdir /root/.kaggle
!cp /content/kaggle.json /root/.kaggle/kaggle.json
!kaggle datasets download -d ivanblch/competetive-course-contest-dataset
!unzip competetive-course-contest-dataset.zip

In [3]:
# path = '/kaggle/input/competative-data-science-course-by-data-feeling/car_train.csv'
path = 'car_train.csv'
car_train = pd.read_csv(path).drop('target_1', axis=1)

features2drop = ['car_id'] 
targets = ['target_2']  
cat_features = ['car_type', 'fuel_type', 'model'] 

filtered_features = [i for i in car_train.columns if (i not in targets and i not in features2drop)] #+ cat_features 
num_features = [i for i in filtered_features if i not in cat_features]

Объявим функцию обучения Catboost с возращением предсказаний на Kfold валидации

In [25]:
def fit_catboost(trial, train, val):
    X_train, y_train = train
    X_val, y_val = val

    param = {
        #"objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.01),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 2, 17),
        #"colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        'auto_class_weights': trial.suggest_categorical('auto_class_weights', ['SqrtBalanced', "Balanced", 'None']),
        "depth": trial.suggest_int("depth", 6, 12),
        #"boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "12gb",
    }


    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    clf = CatBoostClassifier(**params)

    clf.fit(X=X_train, y=y_train, eval_set=(X_val, y_val), verbose = 250, plot = False)

    # инференс модели
    # y_pred = np.zeros((X_val.shape[0], 9)) # массив для записи финального результата
    y_pred = clf.predict(X_val)#[:,1]
    return clf, y_pred

Напишем функцию **objective** в которую поместим Kfold валидацию, чтобы подбирать лучшие гиперпараметры на всем датасете

In [32]:
from sklearn.metrics import accuracy_score

def objective(trial, return_models=False):
    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    X_train = car_train[filtered_features].drop(targets, axis=1, errors='ignore')
    y_train = car_train[targets]

    scores = []
    models = []
    for train_idx, valid_idx in kf.split(X_train):
        train_data = X_train.iloc[train_idx,:], y_train.iloc[train_idx]
        valid_data = X_train.iloc[valid_idx,:], y_train.iloc[valid_idx]

        model, y_pred = fit_catboost(trial, train_data, valid_data)
        scores.append(accuracy_score(y_pred, valid_data[1]))
        models.append(model)
        
    result = np.mean(scores)
    if return_models:
        return result, models
    else:
        return result

Запускаем Optuna!

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150)

Посмотрим на лучшие параметры

In [ ]:
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

Обучим итоговые модели уже на них

In [ ]:
valid_scores, models = objective(optuna.trial.FixedTrial(study.best_params), return_models=True)

## Визуализация

Чтобы посмотреть всю историю обучения можно вывести ее в виде датафрейма

In [ ]:
trials_df = study.trials_dataframe()
trials_df

Визуализация оптимизаций

In [ ]:
optuna.visualization.plot_optimization_history(study)

## Pruning

Pruning в Optuna — возможно останавливать оптимизацию параметров когда обучающая кривая становится хуже прошлых лучшых результатов.

![Картинка](https://optuna.org/assets/img/pruning-example-with-caption.png)

К сожалению, для Catboost'a нет встроенной в Оптуну функции, но есть для большинтва известных ML фреймворков. Например, таких как XGBoost и LightGBM.

*   XGBoost: `optuna.integration.XGBoostPruningCallback`
*   LightGBM: `optuna.integration.LightGBMPruningCallback`



Для использования Pruning в Catboost можно использовать одну из общих вариаций, таких как `MedianPruner`.
Функция прунинга просто добавляется в функцию `create_study` в параметр `pruner`

In [ ]:
study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials=150)

In [ ]:
# Вопросы
# - В каком из примеров ошибка (например минимизация метрики)
# - Выбрать наиболее правильный range из предоставленных параметров
# - Интерактив с кодом: дать поподбирать параметры, чтобы достичь какую-то точность на датасете